# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Note: pandas and numpy are already imported as pd and np
# The following tables are loaded as pandas DataFrames with the same names: fct_guest_spending
# Please print your final result or dataframe


# Ensure visit_date is datetime
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

# Filter for July 2024
start_date = '2024-07-01'
end_date = '2024-07-31'
july_data = fct_guest_spending[
    (fct_guest_spending['visit_date'] >= start_date) &
    (fct_guest_spending['visit_date'] <= end_date)
]

# Calculate average spending per guest per visit
avg_spending = (
    july_data
    .groupby(['park_experience_type', 'guest_id', 'visit_date'])['amount_spent']
    .sum()  # total spending per guest per visit
    .groupby('park_experience_type')  # now group by experience type
    .mean()
    .reset_index(name='avg_spending_per_guest_per_visit')
)

# Ensure all park_experience_types are included (even with no transactions)
all_types = pd.DataFrame({'park_experience_type': fct_guest_spending['park_experience_type'].unique()})
final_result = all_types.merge(avg_spending, on='park_experience_type', how='left').fillna(0.0)

print(final_result)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# Ensure visit_date is datetime
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

# Filter for August 2024
start_date = '2024-08-01'
end_date = '2024-08-31'
august_data = fct_guest_spending[
    (fct_guest_spending['visit_date'] >= start_date) &
    (fct_guest_spending['visit_date'] <= end_date)
]

# Total spending per guest per visit date
daily_spend = (
    august_data
    .groupby(['guest_id', 'visit_date'])['amount_spent']
    .sum()
    .reset_index()
)

# Guests with more than one visit in August
multi_visit_guests = daily_spend['guest_id'].value_counts()
multi_visit_guests = multi_visit_guests[multi_visit_guests > 1].index
daily_spend = daily_spend[daily_spend['guest_id'].isin(multi_visit_guests)]

# Get first and last visit spending
first_last_spend = (
    daily_spend.sort_values(['guest_id', 'visit_date'])
    .groupby('guest_id')
    .agg(
        first_visit_spending=('amount_spent', 'first'),
        last_visit_spending=('amount_spent', 'last')
    )
    .reset_index()
)

# Calculate spending difference (last - first)
first_last_spend['spending_difference'] = (
    first_last_spend['last_visit_spending'] - first_last_spend['first_visit_spending']
)

print(first_last_spend)

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
# Ensure visit_date is datetime
fct_guest_spending['visit_date'] = pd.to_datetime(fct_guest_spending['visit_date'])

# Filter for September 2024
start_date = '2024-09-01'
end_date = '2024-09-30'
sept_data = fct_guest_spending[
    (fct_guest_spending['visit_date'] >= start_date) &
    (fct_guest_spending['visit_date'] <= end_date)
]

# Calculate total spending per guest in September
guest_spending = (
    sept_data
    .groupby('guest_id')['amount_spent']
    .sum()
    .reset_index(name='total_spent')
)

# Exclude guests with zero spending
guest_spending = guest_spending[guest_spending['total_spent'] > 0]

# Categorize spending levels
def categorize_spending(amount):
    if amount < 50:
        return 'Low'
    elif amount < 100:
        return 'Medium'
    else:
        return 'High'

guest_spending['spending_category'] = guest_spending['total_spent'].apply(categorize_spending)

print(guest_spending)

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)